In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re
from collections import Counter
from fuzzywuzzy import fuzz

# some look up table constructing

In [2]:
thailand = pd.read_excel('thailand_map.xlsx')
eng_thai_province = pd.read_excel('eng_thai_province.xlsx')
eng_thai_province.head()

,Province (changwat),จังหวัด
0,Krabi,กระบี่
1,Krabi,กระบี่
2,Krabi,กระบี่
3,Krabi,กระบี่
4,Krabi,กระบี่


In [3]:
eng_thai_dict = {}
for index in eng_thai_province.index:
    eng_thai_dict[eng_thai_province.loc[index, 'Province (changwat)']] = eng_thai_province.loc[index, 'จังหวัด']
eng_thai_dict['Bangkok'] = 'กรุงเทพมหานคร'
eng_thai_dict['Nongbua Lamphu'] = 'หนองบัวลำภู'
eng_thai_dict['Phangnga'] = 'พังงา'
eng_thai_dict['Prachinburi'] = 'ปราจีนบุรี'
eng_thai_dict['Si Saket'] = 'ศรีสะเกษ'
eng_thai_dict['Singburi'] = 'สิงห์บุรี'
eng_thai_dict['Suphanburi'] = 'สุพรรณบุรี'

### dictionary from zipcode to province

In [4]:
import math

df = pd.DataFrame()
province = 'Amnat Charoen'
count = 0
for index in thailand.index:
    df1_dict = thailand.loc[index, 'Province':'Postcode'].to_dict()
    
    if str(df1_dict['Province']) == 'nan':
        df1_dict['province_in_thai'] = eng_thai_dict[province]
        df1_dict['Province'] = province
        if not math.isnan(df1_dict['Postcode']):
            df1 = pd.DataFrame(df1_dict, index=[str(df1_dict['Postcode'])] )
            df = pd.concat([df1,df])
            count += 1
    elif str(df1_dict['Province']) == 'Province':
        pass
    else:
        province = df1_dict['Province']
        df1_dict['province_in_thai'] = eng_thai_dict[province]
        if not math.isnan(df1_dict['Postcode']):
            df1 = pd.DataFrame(df1_dict, index=[ str(df1_dict['Postcode']) ] )
            df = pd.concat([df1, df])
        
        count += 1


In [5]:
df.to_excel('thailand_map.xls')

In [6]:
df.head()

,District,Postcode,Province,province_in_thai
35120,Thai Charoen,35120,Yasothon,ยโสธร
35170,Sai Mun,35170,Yasothon,ยโสธร
35150,Pa Tio,35150,Yasothon,ยโสธร
35130,Maha Chana Chai,35130,Yasothon,ยโสธร
35120,Loeng Nok Tha,35120,Yasothon,ยโสธร


### reading dataset

In [7]:
raw_txt = pd.read_excel('cleaned_up_raw_txt.xls')
raw_txt.head()

,AddressLine1,AddressLine2,AddressLine3,CustomerConfirmationNumber,Recipient email,Recipient name,Recipient phone,Recipient state,Recipient zipcode,text,อำเภอ District
10309,2 ห้อง259/1 ชั้น2 (ปัทมาคลินิค) เซ็นทรัลแอร์พอ...,ิดล ต.หายยา,NaN,TDPSE09201715809,,เพชรรัตน์,835709003,เชียงใหม่,50100,เพชรรัตน์ กิตตินุสรณ์\n2 fins250/1 ul (บัทมาคล...,เมืองเชียงใหม่
162,54 ปากซอยอ่อนนุช สุขุมวิท 77 แขวงพระโขนงเหนือ ...,NaN,NaN,TDPSL06201700953,,กิตติโฟโต้,23110797,กรุงเทพ,10260,NAN 3\nAN... ...,วัฒนา
1384,บ.เอสแอนด์เจ อินเตอร์เนชั่นแนล อินเดอร์ไพร 600...,องขาม,NaN,TDPSE04201708179,,รัตนาวลี,952507257,ชลบุรี,20230,วรลลลลล์\nก | Tel. 3\n\nรัตนาวลี อารีกุล (65)\...,ศรีราชา
1031,93/119 ซอยจรัญสนิทวงศ์ 25 แขวงบางขุนศรี,NaN,NaN,TDPSE04201708540,,กัลยรัตน์ นิยมจ้อย,909769895,กรุงเทพ,10700,คุณกัลยรัตน์ นิยมจ้อย\n\n93/119 ซ.จรัญสนิทวงศ์...,บางกอกน้อย
10255,609/223 มบ.เดอะคอร์นเนอร์ ม.7 ต.ท่าตูม,NaN,NaN,TDPSE09201714969,,กัลยาณี,872561391,ปราจีนบุรี,25140,“21:\nคุณกัดยาณี รุมพล\n\n609/223 ภูบ้านเดอะคอ...,ศรีมหาโพธิ


In [8]:
# # # created a dataset that use postcode as index
thai_province_list = set(df.loc[:,'province_in_thai'])
len(thai_province_list) ## it have 77 province

77

In [17]:
# first two digit represent province, except for bangkok.
# build the dictionary for lookup
post2pro = {}
for index in df.index:
    try:
        # bangkok and other province have mixed postcode
        if str(df.at[str(index), 'Postcode'][0])[:2] != '10':
            post2pro[str(df.at[str(index), 'Postcode'][0])[:2]] = df.at[str(index), 'province_in_thai'][0]
        else:
            post2pro[str(df.at[str(index), 'Postcode'][0])] = df.at[str(index), 'province_in_thai'][0]
    except Exception:
        post2pro[str(df.at[str(index), 'Postcode'])[:2]] = df.at[str(index), 'province_in_thai']
        
# add postcode 10280 to dictionary, because it is not in the dataset
post2pro['10280'] = 'สมุทรปราการ'
post2pro['10270'] = 'สมุทรปราการ'
post2pro['10130'] = 'กรุงเทพมหานคร'
post2pro['10200'] = 'กรุงเทพมหานคร'
post2pro['10800'] = 'กรุงเทพมหานคร'
post2pro['10500'] = 'กรุงเทพมหานคร'

In [18]:
## building up pro2post
pro2post = {}
for index in df.index:
    try:
        pro2post[ df.at[str(index), 'province_in_thai'][0] ] = str(df.at[str(index), 'Postcode'][0])
    except Exception:
        pro2post[ df.at[str(index), 'province_in_thai'] ] = str(df.at[str(index), 'Postcode'])
pro2post['สมุทรปราการ'] = '10280'
pro2post['กรุงเทพมหานคร'] = '10200'

In [21]:
post2pro

{'10': 'กรุงเทพมหานคร',
 '10100': 'กรุงเทพมหานคร',
 '10110': 'กรุงเทพมหานคร',
 '10120': 'กรุงเทพมหานคร',
 '10130': 'กรุงเทพมหานคร',
 '10140': 'กรุงเทพมหานคร',
 '10150': 'กรุงเทพมหานคร',
 '10160': 'กรุงเทพมหานคร',
 '10170': 'กรุงเทพมหานคร',
 '10200': 'กรุงเทพมหานคร',
 '10210': 'กรุงเทพมหานคร',
 '10220': 'กรุงเทพมหานคร',
 '10230': 'กรุงเทพมหานคร',
 '10240': 'กรุงเทพมหานคร',
 '10250': 'กรุงเทพมหานคร',
 '10260': 'กรุงเทพมหานคร',
 '10270': 'สมุทรปราการ',
 '10280': 'สมุทรปราการ',
 '10310': 'กรุงเทพมหานคร',
 '10400': 'กรุงเทพมหานคร',
 '10500': 'กรุงเทพมหานคร',
 '10510': 'กรุงเทพมหานคร',
 '10540': 'สมุทรปราการ',
 '10600': 'กรุงเทพมหานคร',
 '10700': 'กรุงเทพมหานคร',
 '10800': 'กรุงเทพมหานคร',
 '11': 'นนทบุรี',
 '12': 'ปทุมธานี',
 '13': 'พระนครศรีอยุธยา',
 '14': 'อ่างทอง',
 '15': 'ลพบุรี',
 '16': 'สิงห์บุรี',
 '17': 'ชัยนาท',
 '18': 'สระบุรี',
 '20': 'ชลบุรี',
 '21': 'ระยอง',
 '22': 'จันทบุรี',
 '23': 'ตราด',
 '24': 'ฉะเชิงเทรา',
 '25': 'ปราจีนบุรี',
 '26': 'นครนายก',
 '27': 'สระแก้ว',
 '30': 'น

## necessary functions to get the province and the postcode

In [24]:
def get_province_postcode(text, index):
    if len(str(text)) < 10:
        print ('text too short', index)
    postcodes = [str(int(float(s[:5]))) for s in re.findall(r'-?\d+\.?\d*', text) if len(str(int(float(s[:5]))))>=5 and int(float(s[:5])) >= 0]
    province_list = []
    for postcode in postcodes:
        try:
            pre_province = df.at[str(postcode),'province_in_thai'][0] if len(df.at[str(postcode),'province_in_thai'][0]) > 1 else df.at[str(postcode),'province_in_thai']
            province_list.append([pre_province,postcode]) # special indexing to get the value
            
            
        except Exception as e:
            if postcode[:2] != '10':
                province_list.append([post2pro[postcode],postcode]) ## deleted [:2] here because might be other except bangkok
            else:
                province_list.append([post2pro[postcode],postcode])

    return province_list
            
def get_province_levenshtein(text):
   
    words = text.split()[-4:]
    final_dict = {}
    words = [i for i in words if len(i) >= 4]
    dict_score = defaultdict(lambda: 0)
    for word in words:
        # check the correct word that have a lot of common with the word
        purged_word = purge(word)
        # loop through all the states and see which one get higher accuracy       
        

        for province in thai_province_list:
            purged_province = purge(province)
            
            score = fuzz.partial_ratio(purged_word, purged_province)
            dict_score[province] = max(score, dict_score[province])
        
        correct_province = {k:v for k,v in dict_score.items() if v == max(dict_score.values())}
        final_dict = {**final_dict, **correct_province}
    return  list( {k:v for k,v in final_dict.items() if v == max(final_dict.values())}.keys() )
        
            
def purge(string):
    new_string = ''
    delete_list = [b'\xe0\xb8\xb1',b'\xe0\xb8\xb3',b'\xe0\xb8\xb4',b'\xe0\xb8\xb5',b'\xe0\xb8\xb6',b'\xe0\xb8\xb7',b'\xe0\xb8\xb8',b'\xe0\xb8\xb9',b'\xe0\xb8\xba',b'\xe0\xb9\x87',b'\xe0\xb9\x88',b'\xe0\xb9\x89',b'\xe0\xb9\x8a',b'\xe0\xb9\x8b',b'\xe0\xb9\x8c',b'\xe0\xb9\x8d',b'\xe0\xb9\x8e']
    for char in string:
        if char.encode('utf') not in delete_list:
            new_string += char
    return new_string

In [25]:
index = 10320
text = raw_txt.get_value(index,'text')
get_province_postcode(text, index)

/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


[['กรุงเทพมหานคร', '10310']]

## 1. identify postcode and provinces and purge them off the text.

In [149]:
df_po = pd.DataFrame() ## if get province convert back to postcode
 ## if get postcode convert back to province
for index in raw_txt.index:
    
    text = raw_txt.get_value(index,'text')
    
    # convert the whole text into dictionary
    mydict = raw_txt.loc[index]
    
    getPostCode = get_province_postcode(text, index)
    ### result is province
    if  getPostCode[0] == []:
        
        text = raw_txt.get_value(index,'text')
        pre_province = get_province_levenshtein(text)
        ## get the province
        mydict['predicted_province'] = pre_province
        
        
        ## get the postcode through lookup dict
        ### pro2post does not cover all the cases ## fix this
        
        try:
            pre_postcode = pro2post[pre_province]
        except Exception:
            pre_postcode = 'could not find postcode through province'
        mydict['predicted_postcode'] = pre_postcode
            
        ## purge the postcode, usually not necessary because the postcode is usually not contained there
        try:
            if pro2post[pre_province] in text:
                text = text.replace(pre_postcode, ' ')
        except Exception:
            pass
            
        ## purge province
        try:
            if pre_province in text:
                text = text.replace(pre_province, ' ')
        except:
            pass
        
    ### result is postcode    
    elif len(get_province_postcode(text, index)) > 1:
        # chooose which one have a lower levenshtein value compare to the province get in province levenshtein
        # assuming there is gonna be one province levenshtein
        # if we have multiple postcode
        max_score = 0
        
        leven = get_province_levenshtein(text)
        
        ## compare and get the one that have max value for input
        for p in postcode:
            for l in leven:
                max_score = max(max_score, fuzz.partial_ratio(p[0], l))
        try:        
            for p in postcode:
                for l in leven:
                    if fuzz.partial_ratio(p[0], l) == max_score:
                        pre_province = p[0]
                        pre_postcode = p[1]
                        raise StopIteration
        except StopIteration:
            pass
    
        
        mydict['predicted_postcode'] = pre_postcode

        ## purge postcode
        if pre_postcode in text:
            text = text.replace(pre_postcode, ' ')

            
        ## purge province
        try:
            if pre_province in text:
                text = text.replace(pre_province, ' ')
        except:
            pass
            
    ### result is postcode
    else:
        try:
            text = raw_txt.get_value(index,'text')
            pre_province = get_province_postcode(text,index)[0][0]
            pre_postcode = get_province_postcode(text,index)[0][1]
        except Exception:
            pre_province = 'error'
            
        ## get the province
        mydict['predicted_province'] = pre_province
        
        ## get the postcode
        mydict['predicted_postcode'] = pre_postcode
            
        ## purge postcode
        try:
            if pro2post[pre_province] in text:
                text = text.replace(pre_postcode, ' ')
        except Exception:
            pass
            
        ## purge province
        try:
            if pre_province in text:
                text = text.replace(pre_province, ' ')
        except:
            pass
            
    mydict['text'] = text
    df = pd.DataFrame(mydict)
    df_po = pd.concat([df_po, df], axis=1)

10309
162


/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/i351707/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:769: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  s

1384
1031
10255
10241
10269
10296
1596
1597
10297
10268
10240
10254
10693
1420
10308


/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

10320
10322
1620
10242
10256
1224
10295
10294
1543
10243
1153
1621
1609
1423
10323
1427
10327
10253
1590
10290
10291
10285
10252
10508
1618
1397
1383
10440
1426
1430
10318
1356
1418
1381
10278
10250
10244
10293
10287
10286
10292
1592
1586
10251
1545
1182
1196
1419
10325
1425
1318
1330
1442
10829
1536
1251
10237
1084
1047
1721
1709
10155
10157
10779
1086
1087
1093
1736
10778
1078
1326
1040
1726
1083
10783
10224
10557
10225
1069
10347
1309
10151
1725
1719
10780
1269
1095
1718
1724
10622
1334
10439
1305
1073
1259
1489
1304
1462
1070
10214
1272
1266
1299
1298
1071
1465
1505
1538
10238
1074
10364
1464
1076
1738
1506
1512
1088
10777
130
1602
10704
10506
10248
10249
1549
10507
10275
10705
1603
10315
10317
1365
1417
1601
10511
10288
10289
10510
10100
10312
10266
10272
10299
10273
10267
1611
1605
10307
10313
1613
10067
10517
10271
10265
10503
10516
10270
1202
10066
10310


In [150]:
df_po = df_po.T
df_po.to_excel('raw_text.xls')

In [151]:
import string
# how long is the digit, it is used for tel
def digit_len(string):
    count = 0
    for i in string:
        if i.isdigit():
            count += 1
    return count

# purge the data to avoid unecessary changes
def purge(string):
    new_string = ''
    delete_list = [b'\xe0\xb8\xb1',b'\xe0\xb8\xb3',b'\xe0\xb8\xb4',b'\xe0\xb8\xb5',b'\xe0\xb8\xb6',b'\xe0\xb8\xb7',b'\xe0\xb8\xb8',b'\xe0\xb8\xb9',b'\xe0\xb8\xba',b'\xe0\xb9\x87',b'\xe0\xb9\x88',b'\xe0\xb9\x89',b'\xe0\xb9\x8a',b'\xe0\xb9\x8b',b'\xe0\xb9\x8c',b'\xe0\xb9\x8d',b'\xe0\xb9\x8e']
    for char in string:
        if char.encode('utf') not in delete_list:
            new_string += char
    return new_string

def have_digit(string):
    return any(char.isdigit for char in string)

def get_rid_punctuation(st):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in st if ch not in exclude)
    return s

In [152]:
names = ['น้อง','พี่','คุณ','นาย','นาง','นางสาว','คุณหนู','คุณหญิง']

In [153]:
# clean up the text

In [154]:
cleaned_up_list = ['Tel', 'ผรบ', 'กรณาสง', 'รหสไปรษณย','ชอผรบ']

In [155]:
## process the name first

In [156]:
raw_txt = pd.read_excel('raw_text.xls')

In [157]:
df_name = pd.DataFrame()
for index in raw_txt.index:
    mydict = raw_txt.loc[index, :]
    text = raw_txt.at[index,'text']
    text_list= raw_txt.at[index, 'text'].split('\n')
    name = 'unable to identify name'
    
    for word in text_list:
        word = get_rid_punctuation(word)
        if len(word.strip()) > 3 and not any(i in purge(word) for i in cleaned_up_list) and digit_len(word) < 3:
#             print(word, raw_txt.at[index, 'Recipient name'])
            name = word
            break
    purged_text = text.replace(word,' ')
    mydict['text'] = purged_text
    mydict['predicted_name'] = name
    df = pd.DataFrame(mydict)
    df_name = pd.concat([df, df_name],axis = 1)

/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/i351707/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:769: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [158]:
df_name = df_name.T
df_name.to_excel('name.xls')

# telephone numbers

In [160]:
name = pd.read_excel('name.xls')

In [161]:
df_num = pd.DataFrame()
for index in name.index:
    text_list = name.at[index, 'text'].split()
    text = name.at[index, 'text']
    
    text_num = sorted(text_list,key=digit_len)[-1]
    
    num = 'unable to identify num'
    if len(text_num) > 7:
        num = text_num
        
    mydict = name.loc[index]
    purged_text = text.replace(num, ' ')
    mydict['left_over'] = purged_text
    mydict['predicted_tel'] = num
    df = pd.DataFrame(mydict)
    df_num = pd.concat([df_num, df], axis=1)
    
    

/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/i351707/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:769: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/Users/i351707/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [162]:
df_num = df_num.T
df_num.to_excel('num.xls')